In [1]:
!pip install langchain
!pip install langchain-community
!pip install pymupdf
!pip install faiss-cpu
!pip install transformers
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# 단계 1: 문서 로드(Load Documents)
url = 'https://v.daum.net/v/20250516173007356'
loader = WebBaseLoader(url)
docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embeddings = HuggingFaceEmbeddings(
        model_name="BM-K/KoSimCSE-roberta-multitask"
        # model_name = 'jhgan/ko-sroberta-nli'
    )

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """다음 컨텍스트를 사용하여 질문에 답변하세요.
    답변은 간결하고 정확해야 하며, 한국어로 작성하세요.
    컨텍스트에 없는 정보는 사용하지 마세요.

#Context:
{context}

#Question:
{question}

#Answer:"""
)

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
model_name = "beomi/Llama-3-Open-Ko-8B-Instruct-preview"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    device=0,  # CPU 사용, GPU 사용 시 0 또는 적절한 디바이스 ID
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

<ipython-input-2-9eae86f5621b>:21: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/764 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/143 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-2-9eae86f5621b>:64: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


In [3]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "14일 기준 이더리움 가격은 얼마나 올랐지?"
response = chain.invoke(question)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


다음 컨텍스트를 사용하여 질문에 답변하세요.
    답변은 간결하고 정확해야 하며, 한국어로 작성하세요.
    컨텍스트에 없는 정보는 사용하지 마세요.

#Context:
[Document(id='a47c873a-c698-4b57-962c-4445f1b8a96c', metadata={'source': 'https://v.daum.net/v/20250516173007356', 'title': '"비트코인 뛰어넘는 잠재력"… 이더리움 심상찮은 질주', 'language': 'No language found.'}, page_content='코인시장에서 비트코인이 황제라면 이더리움은 왕세자다. 시가총액이 비트코인에 이어 2위인 데다 스마트 콘트랙트 등 블록체인을 활용한 각종 기술 측면에서도 우수하다는 평을 받기 때문이다. 한때 미래의 성장 잠재력은 비트코인을 능가한다는 평가를 받기도 했다.\n한동안 잠잠하던 이더리움의 최근 상승세가 심상치 않다. 일주일 만에 이더리움 값이 40% 이상 급등하면서 비트코인의 상승세를 훌쩍 뛰어넘었다. 특히 최근 급등이 기술적인 부분이나 자금 수급 측면에서 이유 있는 상승이라는 지적이 제기되면서 투자자들의 관심이 집중되고 있다.\n비트코인보다 5배 더 올랐다\n5월 들어 미·중 관세협상 등으로 코인시장에 훈풍이 부는 가운데 이더리움이 타 코인 대비 유독 높은 상승세를 기록 중이다. 코인시장이 강세를 보였던 14일 기준 이더리움 가격은 전주 대비 46.7% 올라 374만4000원대를 나타내고 있다. 비트코인이 8%, XRP와 솔라나가 약 20% 상승한 것에 비해 2~5배 높다. 이더리움이 급등한 주된 이유로는 먼저 기술적인 측면이 꼽힌다. 지난 7일 이더리움의 펙트라 업그레이드가 완료됐다. 펙트라는 비트코인과 동일한 작업증명(PoW) 기반 채굴에서 금융 시스템과 유사한 지분증명(PoS) 방식으로 합의 알고리즘을 변경하면서 이더리움 2.0 시대를 열었다고 평가받는 머지 업그레이드 이후 대대적인 변화다.\nPoS 방식으로 구현된 스테이킹의 

In [4]:
question = "이더리움 값은 사상 최고가 대비 몇 %지?"
response = chain.invoke(question)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


다음 컨텍스트를 사용하여 질문에 답변하세요.
    답변은 간결하고 정확해야 하며, 한국어로 작성하세요.
    컨텍스트에 없는 정보는 사용하지 마세요.

#Context:
[Document(id='4cff9236-0043-4752-b3ac-04bd2ea4050d', metadata={'source': 'https://v.daum.net/v/20250516173007356', 'title': '"비트코인 뛰어넘는 잠재력"… 이더리움 심상찮은 질주', 'language': 'No language found.'}, page_content='솔라나, 수이 대비 기술적 우월성 확인\n또 펙트라 업그레이드가 솔라나, 수이 등 경쟁자에 비해 이더리움의 기술적 우월성이 높음을 증명하는 사례로 받아들여지면서 기관투자자들에게서 호평을 받고 있다. 블록체인 분석 기업인 메사리의 노아 데이비스 리서치 총괄은 "펙트라 업그레이드는 이더리움이 글로벌 금융 인프라로 도약하기 위한 중요한 전환점"이라며 "스테이킹 한도 확대는 기관투자자들의 참여 장벽을 낮추는 요인이 될 것"이라고 밝혔다.\n최근 불고 있는 실물자산토큰화(RWA) 추세에서도 이더리움의 기술적 우월성이 실제 사례로 이어질 것이라는 전망이 많다. 이미 JP모건, 골드만삭스 등은 자사의 디지털 자산 플랫폼을 이더리움 기반으로 구축하고 있다. 블랙록의 첫 토큰화 펀드인 비들(BUIDL)도 이더리움에서 가장 먼저 발행된 바 있다.\n이더리움 수요가 늘어나면서 향후 가격 전망도 긍정적이다. 시카고 상품거래소(CME)의 이더리움 옵션 시장에서는 올해 말 이더리움 가격을 1만달러로 예상하는 콜옵션 거래량이 최근 큰 폭으로 증가했다. 기술적 우월성에 기반한 강세에 베팅하는 투자자들이 그만큼 늘어난 것으로 해석된다.\n새 탈중앙화 서비스 나와야 체질 개선\n최근 코인시장 전반에 훈풍이 불고 있음을 감안할 때 이더리움이 저가 매력으로 단기 급등한 것뿐이라는 분석도 제기된다. 비트코인이 올 1월 사상 최고가를 경신하는 등 3년간 최대 7배나 